# 🧲 Tutorial: Benchmark del Levitador Magnético

## Optimización con Algoritmos Bio-Inspirados y Metaheurísticas

**Autor:** Jesús  
**Versión:** 1.0  
**Fecha:** Diciembre 2024

---

### 🎯 Objetivo de este Tutorial

Este notebook te guiará paso a paso para:

1. **Entender** el problema físico del levitador magnético
2. **Instalar** el benchmark desde GitHub
3. **Usar** la función de fitness para evaluar soluciones
4. **Implementar** tu propio algoritmo metaheurístico
5. **Comparar** resultados con otros algoritmos

---

## 🎬 Videos Explicativos\n\nAntes de comenzar, te recomendamos ver estos videos cortos que explican el problema:\n\n| Video | Descripción | Duración |\n|-------|-------------|----------|\n| 1. Problema Físico | Qué es el levitador y su modelo | ~1 min |\n| 2. Función de Fitness | Cómo evaluar soluciones | ~1 min |\n| 3. Cómo Optimizar | El ciclo de optimización | ~1 min |

In [ ]:
# Cargar y mostrar videos desde GitHub
# (Descomentar las líneas según el video que quieras ver)

from IPython.display import Video, display
import os

# URL base del repositorio (actualizar con tu usuario de GitHub)
GITHUB_USER = "JRavenelco"  # <-- Usuario actualizado
BASE_URL = f"https://raw.githubusercontent.com/{GITHUB_USER}/levitador-benchmark/main/videos/"

# Función para mostrar video
def mostrar_video(nombre):
    """Muestra un video desde GitHub o localmente."""
    # Primero intenta local
    local_path = f"videos/{nombre}"
    if os.path.exists(local_path):
        display(Video(local_path, embed=True, width=640))
    else:
        # Si no existe local, intenta desde GitHub
        url = BASE_URL + nombre
        display(Video(url, embed=True, width=640))

# Descomentar para ver cada video:
# mostrar_video("01_problema_fisico.mp4")
# mostrar_video("02_funcion_fitness.mp4")
# mostrar_video("03_como_optimizar.mp4")

## 📚 Parte 1: El Problema Físico

### ¿Qué es un Levitador Magnético?

Es un sistema donde una **esfera de acero** se mantiene suspendida en el aire mediante un **electroimán**. La posición de la esfera se controla variando la corriente eléctrica.

![Diagrama del Levitador](https://upload.wikimedia.org/wikipedia/commons/thumb/6/6d/Magnetic_levitation.svg/200px-Magnetic_levitation.svg.png)

### El Modelo Matemático

La inductancia del electroimán varía con la distancia según:

$$L(y) = k_0 + \frac{k}{1 + y/a}$$

Donde:
- $y$ = posición de la esfera (metros)
- $k_0$ = inductancia base (Henrios)
- $k$ = coeficiente de inductancia (Henrios)
- $a$ = parámetro geométrico (metros)

### El Problema de Optimización

**Objetivo:** Encontrar los valores óptimos de $[k_0, k, a]$ que minimicen el error entre:
- La simulación del modelo matemático
- Los datos experimentales reales

**Función Objetivo:**
$$\text{fitness}(k_0, k, a) = \text{MSE} = \frac{1}{N}\sum_{i=1}^{N}(y_{real,i} - y_{sim,i})^2$$

**Espacio de Búsqueda:**

| Variable | Límite Inferior | Límite Superior |
|----------|-----------------|------------------|
| $k_0$    | 0.0001          | 0.1              |
| $k$      | 0.0001          | 0.1              |
| $a$      | 0.0001          | 0.05             |

---

## 🔧 Parte 2: Instalación

### Opción A: Clonar desde GitHub (Recomendado)

In [ ]:
# Descomentar y ejecutar para clonar el repositorio
# !git clone https://github.com/TU-USUARIO/levitador-benchmark.git
# %cd levitador-benchmark

### Opción B: Descargar directamente (si ya tienes los archivos)

In [ ]:
# Si estás en Google Colab, descarga los archivos necesarios:
# !wget https://raw.githubusercontent.com/TU-USUARIO/levitador-benchmark/main/levitador_benchmark.py
# !wget https://raw.githubusercontent.com/TU-USUARIO/levitador-benchmark/main/data/datos_levitador_20251024_162706.txt -P data/

### Instalar Dependencias

In [ ]:
# Instalar dependencias necesarias
!pip install numpy scipy pandas matplotlib --quiet

---

## 🚀 Parte 3: Usando el Benchmark

### 3.1 Importar y Crear el Problema

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from levitador_benchmark import LevitadorBenchmark

# Crear instancia del benchmark
# Sin argumentos: usa datos sintéticos (para pruebas rápidas)
# Con ruta: usa datos experimentales reales
problema = LevitadorBenchmark()  # Datos sintéticos
# problema = LevitadorBenchmark('data/datos_levitador_20251024_162706.txt')  # Datos reales

print(problema)

### 3.2 Entender la Interfaz del Benchmark

In [ ]:
# Propiedades importantes del problema
print("=" * 50)
print("PROPIEDADES DEL PROBLEMA")
print("=" * 50)

print(f"\n📐 Dimensión: {problema.dim}")
print(f"📊 Número de muestras: {len(problema.t_real)}")

print(f"\n🔍 Espacio de búsqueda:")
for name, (lb, ub) in zip(problema.variable_names, problema.bounds):
    print(f"   {name}: [{lb}, {ub}]")

print(f"\n📌 Solución de referencia: {problema.reference_solution}")

### 3.3 Evaluar una Solución Candidata

Esta es la función principal que usarás en tu algoritmo:

In [ ]:
# Evaluar una solución específica
solucion = [0.036, 0.0035, 0.005]  # [k0, k, a]

# La función fitness retorna el MSE (Error Cuadrático Medio)
error = problema.fitness_function(solucion)

print(f"Solución: {solucion}")
print(f"Error (MSE): {error:.6e}")
print(f"\n💡 Menor error = Mejor solución")

### 3.4 Explorar el Espacio de Búsqueda

In [ ]:
# Evaluar varias soluciones aleatorias para entender el paisaje de fitness
np.random.seed(42)

n_muestras = 100
soluciones = []
errores = []

print("Explorando espacio de búsqueda...")
for i in range(n_muestras):
    # Generar solución aleatoria dentro de los límites
    sol = [np.random.uniform(lb, ub) for lb, ub in problema.bounds]
    err = problema.fitness_function(sol)
    
    soluciones.append(sol)
    errores.append(err)

errores = np.array(errores)
print(f"\n📈 Estadísticas de {n_muestras} soluciones aleatorias:")
print(f"   Mejor error:  {errores.min():.6e}")
print(f"   Peor error:   {errores.max():.6e}")
print(f"   Error medio:  {errores.mean():.6e}")
print(f"   Desv. est.:   {errores.std():.6e}")

In [ ]:
# Visualizar distribución de errores
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.hist(errores, bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Error (MSE)')
plt.ylabel('Frecuencia')
plt.title('Distribución de Errores')
plt.axvline(errores.min(), color='r', linestyle='--', label=f'Mejor: {errores.min():.2e}')
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(np.log10(errores + 1e-10), bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('log₁₀(Error)')
plt.ylabel('Frecuencia')
plt.title('Distribución de Errores (escala log)')

plt.tight_layout()
plt.show()

---

## 🧬 Parte 4: Implementa Tu Propio Algoritmo

### 4.1 Plantilla Base

Usa esta plantilla para implementar tu metaheurística:

In [ ]:
class MiAlgoritmo:
    """
    Plantilla para implementar tu algoritmo metaheurístico.
    
    TODO: Reemplaza 'MiAlgoritmo' con el nombre de tu algoritmo
    (ej: AlgoritmoGenetico, PSO, ColoniaHormigas, etc.)
    """
    
    def __init__(self, problema, **params):
        """
        Inicializa el algoritmo.
        
        Args:
            problema: Instancia de LevitadorBenchmark
            **params: Parámetros de tu algoritmo
        """
        self.problema = problema
        self.dim = problema.dim
        self.bounds = np.array(problema.bounds)
        self.lb = self.bounds[:, 0]
        self.ub = self.bounds[:, 1]
        
        # TODO: Define los parámetros de tu algoritmo
        self.pop_size = params.get('pop_size', 30)
        self.max_iter = params.get('max_iter', 100)
        
        # Historial para análisis
        self.historia_mejor = []
        self.historia_media = []
        self.evaluaciones = 0
    
    def inicializar_poblacion(self):
        """Genera población inicial aleatoria."""
        return np.random.uniform(
            self.lb, self.ub, 
            size=(self.pop_size, self.dim)
        )
    
    def evaluar(self, individuo):
        """Evalúa un individuo usando la función de fitness."""
        self.evaluaciones += 1
        return self.problema.fitness_function(individuo)
    
    def evaluar_poblacion(self, poblacion):
        """Evalúa toda la población."""
        return np.array([self.evaluar(ind) for ind in poblacion])
    
    def optimizar(self):
        """
        Ejecuta el algoritmo de optimización.
        
        TODO: Implementa la lógica de tu algoritmo aquí.
        
        Returns:
            mejor_solucion: array con [k0, k, a]
            mejor_fitness: error mínimo encontrado
        """
        # 1. Inicializar población
        poblacion = self.inicializar_poblacion()
        fitness = self.evaluar_poblacion(poblacion)
        
        # Guardar el mejor
        mejor_idx = np.argmin(fitness)
        mejor_global = poblacion[mejor_idx].copy()
        mejor_fitness = fitness[mejor_idx]
        
        # 2. Bucle principal
        for iteracion in range(self.max_iter):
            
            # ================================================
            # TODO: IMPLEMENTA TU ALGORITMO AQUÍ
            # ================================================
            # Ejemplo simple: búsqueda aleatoria (reemplaza esto)
            nueva_poblacion = self.inicializar_poblacion()
            nuevo_fitness = self.evaluar_poblacion(nueva_poblacion)
            
            # Mantener los mejores
            for i in range(self.pop_size):
                if nuevo_fitness[i] < fitness[i]:
                    poblacion[i] = nueva_poblacion[i]
                    fitness[i] = nuevo_fitness[i]
            # ================================================
            
            # Actualizar mejor global
            mejor_idx = np.argmin(fitness)
            if fitness[mejor_idx] < mejor_fitness:
                mejor_global = poblacion[mejor_idx].copy()
                mejor_fitness = fitness[mejor_idx]
            
            # Guardar historial
            self.historia_mejor.append(mejor_fitness)
            self.historia_media.append(np.mean(fitness))
            
            # Imprimir progreso cada 10 iteraciones
            if iteracion % 10 == 0:
                print(f"Iter {iteracion:3d}: Mejor = {mejor_fitness:.6e}")
        
        return mejor_global, mejor_fitness
    
    def graficar_convergencia(self):
        """Grafica la curva de convergencia."""
        plt.figure(figsize=(10, 4))
        
        plt.subplot(1, 2, 1)
        plt.plot(self.historia_mejor, 'b-', label='Mejor')
        plt.plot(self.historia_media, 'r--', alpha=0.5, label='Media')
        plt.xlabel('Iteración')
        plt.ylabel('Fitness (MSE)')
        plt.title('Convergencia')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.semilogy(self.historia_mejor, 'b-')
        plt.xlabel('Iteración')
        plt.ylabel('Fitness (log)')
        plt.title('Convergencia (escala log)')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

### 4.2 Probar la Plantilla

In [ ]:
# Probar la plantilla (búsqueda aleatoria por defecto)
mi_algo = MiAlgoritmo(problema, pop_size=20, max_iter=50)

print("Ejecutando algoritmo...\n")
mejor_sol, mejor_err = mi_algo.optimizar()

print(f"\n{'='*50}")
print("RESULTADO FINAL")
print(f"{'='*50}")
print(f"k0 = {mejor_sol[0]:.6f}")
print(f"k  = {mejor_sol[1]:.6f}")
print(f"a  = {mejor_sol[2]:.6f}")
print(f"Error (MSE): {mejor_err:.6e}")
print(f"Evaluaciones totales: {mi_algo.evaluaciones}")

In [ ]:
# Graficar convergencia
mi_algo.graficar_convergencia()

---

## 🔬 Parte 5: Ejemplo Completo - Evolución Diferencial

Aquí tienes un ejemplo completo de un algoritmo implementado:

In [ ]:
class EvolucionDiferencial:
    """
    Implementación de Evolución Diferencial (DE/rand/1/bin).
    
    Referencia: Storn, R., & Price, K. (1997)
    """
    
    def __init__(self, problema, pop_size=30, max_iter=100, F=0.8, CR=0.9):
        self.problema = problema
        self.dim = problema.dim
        self.bounds = np.array(problema.bounds)
        self.lb = self.bounds[:, 0]
        self.ub = self.bounds[:, 1]
        
        # Parámetros DE
        self.pop_size = pop_size
        self.max_iter = max_iter
        self.F = F    # Factor de escala de mutación
        self.CR = CR  # Probabilidad de cruce
        
        self.historia_mejor = []
        self.evaluaciones = 0
    
    def optimizar(self):
        # Inicializar población
        poblacion = np.random.uniform(self.lb, self.ub, (self.pop_size, self.dim))
        fitness = np.array([self.problema.fitness_function(ind) for ind in poblacion])
        self.evaluaciones = self.pop_size
        
        mejor_idx = np.argmin(fitness)
        mejor_global = poblacion[mejor_idx].copy()
        mejor_fitness = fitness[mejor_idx]
        
        for gen in range(self.max_iter):
            for i in range(self.pop_size):
                # Seleccionar 3 individuos distintos
                indices = [j for j in range(self.pop_size) if j != i]
                a, b, c = poblacion[np.random.choice(indices, 3, replace=False)]
                
                # Mutación: v = a + F * (b - c)
                mutante = a + self.F * (b - c)
                mutante = np.clip(mutante, self.lb, self.ub)
                
                # Cruce binomial
                trial = poblacion[i].copy()
                j_rand = np.random.randint(self.dim)
                for j in range(self.dim):
                    if np.random.random() < self.CR or j == j_rand:
                        trial[j] = mutante[j]
                
                # Selección
                trial_fitness = self.problema.fitness_function(trial)
                self.evaluaciones += 1
                
                if trial_fitness < fitness[i]:
                    poblacion[i] = trial
                    fitness[i] = trial_fitness
                    
                    if trial_fitness < mejor_fitness:
                        mejor_global = trial.copy()
                        mejor_fitness = trial_fitness
            
            self.historia_mejor.append(mejor_fitness)
            
            if gen % 10 == 0:
                print(f"Gen {gen:3d}: Mejor = {mejor_fitness:.6e}")
        
        return mejor_global, mejor_fitness

In [ ]:
# Ejecutar Evolución Diferencial
de = EvolucionDiferencial(problema, pop_size=20, max_iter=50, F=0.8, CR=0.9)

print("Ejecutando Evolución Diferencial...\n")
mejor_sol, mejor_err = de.optimizar()

print(f"\n{'='*50}")
print("RESULTADO - Evolución Diferencial")
print(f"{'='*50}")
print(f"k0 = {mejor_sol[0]:.6f}")
print(f"k  = {mejor_sol[1]:.6f}")
print(f"a  = {mejor_sol[2]:.6f}")
print(f"Error (MSE): {mejor_err:.6e}")
print(f"Evaluaciones: {de.evaluaciones}")

In [ ]:
# Graficar convergencia
plt.figure(figsize=(8, 4))
plt.semilogy(de.historia_mejor, 'b-', linewidth=2)
plt.xlabel('Generación', fontsize=12)
plt.ylabel('Mejor Fitness (MSE)', fontsize=12)
plt.title('Convergencia - Evolución Diferencial', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

---

## 📊 Parte 6: Comparar Algoritmos

### 6.1 Protocolo de Comparación

Para una comparación justa, todos los algoritmos deben:
1. Usar el **mismo número de evaluaciones** (ej: 3000)
2. Ejecutarse **múltiples veces** (ej: 30 corridas)
3. Reportar estadísticas: **media, desviación estándar, mejor, peor**

In [ ]:
def comparar_algoritmos(problema, algoritmos, n_corridas=10, max_evals=1000):
    """
    Compara múltiples algoritmos de forma justa.
    
    Args:
        problema: Instancia de LevitadorBenchmark
        algoritmos: Dict {nombre: ClaseAlgoritmo}
        n_corridas: Número de corridas independientes
        max_evals: Máximo de evaluaciones por corrida
    """
    resultados = {}
    
    for nombre, AlgClase in algoritmos.items():
        print(f"\n{'='*50}")
        print(f"Probando: {nombre}")
        print(f"{'='*50}")
        
        errores = []
        for corrida in range(n_corridas):
            # Calcular parámetros para respetar max_evals
            pop_size = 20
            max_iter = max_evals // pop_size
            
            algo = AlgClase(problema, pop_size=pop_size, max_iter=max_iter)
            _, mejor_err = algo.optimizar()
            errores.append(mejor_err)
            print(f"  Corrida {corrida+1}/{n_corridas}: {mejor_err:.6e}")
        
        errores = np.array(errores)
        resultados[nombre] = {
            'media': errores.mean(),
            'std': errores.std(),
            'mejor': errores.min(),
            'peor': errores.max(),
            'todos': errores
        }
    
    return resultados

In [ ]:
# Ejemplo: comparar tu algoritmo vs Evolución Diferencial
# Descomentar cuando tengas tu algoritmo implementado

# algoritmos = {
#     'Mi Algoritmo': MiAlgoritmo,
#     'Evolución Diferencial': EvolucionDiferencial,
# }

# resultados = comparar_algoritmos(problema, algoritmos, n_corridas=5, max_evals=500)

# # Mostrar tabla de resultados
# print("\n" + "="*70)
# print("TABLA DE RESULTADOS")
# print("="*70)
# print(f"{'Algoritmo':<25} {'Media':>12} {'Std':>12} {'Mejor':>12}")
# print("-"*70)
# for nombre, stats in resultados.items():
#     print(f"{nombre:<25} {stats['media']:>12.6e} {stats['std']:>12.6e} {stats['mejor']:>12.6e}")

---

## 📝 Parte 7: Entrega de Resultados

### Formato de Entrega

Por favor, documenta tu algoritmo con la siguiente información:

1. **Nombre del algoritmo**
2. **Parámetros utilizados**
3. **Resultados** (tabla con media, std, mejor, peor)
4. **Gráfica de convergencia**
5. **Código fuente** (tu clase de algoritmo)

In [ ]:
# Plantilla para documentar tus resultados

reporte = """
# REPORTE DE RESULTADOS

## Algoritmo: [NOMBRE DE TU ALGORITMO]

### Parámetros:
- Tamaño de población: []
- Máximo de iteraciones: []
- [Otros parámetros específicos]

### Resultados (30 corridas):
| Métrica | Valor |
|---------|-------|
| Media   | [    ] |
| Std     | [    ] |
| Mejor   | [    ] |
| Peor    | [    ] |

### Mejor solución encontrada:
- k0 = []
- k  = []
- a  = []

### Observaciones:
[Comentarios sobre el comportamiento del algoritmo]
"""

print(reporte)

---

## 🎓 Parte 8: Ideas para Algoritmos

Aquí hay algunas ideas de algoritmos que puedes implementar:

### Algoritmos Clásicos:
- **Algoritmo Genético (GA)** - Selección, cruce, mutación
- **Enjambre de Partículas (PSO)** - Velocidad y posición
- **Evolución Diferencial (DE)** - Ya implementado como ejemplo
- **Estrategias Evolutivas (ES)** - (μ+λ) o (μ,λ)

### Algoritmos Inspirados en la Naturaleza:
- **Colonia de Hormigas (ACO)**
- **Colonia de Abejas (ABC)**
- **Algoritmo del Murciélago (BA)**
- **Búsqueda del Cuco (CS)**
- **Algoritmo del Lobo Gris (GWO)**
- **Optimización de la Ballena (WOA)**

### Algoritmos Híbridos:
- Combina características de varios algoritmos
- Ejemplo: PSO con mutación diferencial

---